## Wave equation in two dimensions

### The equation
In two dimensions, the wave equation will read

$\frac{\partial^2 u}{\partial t^2} = \nu^2 \nabla^2 u$

where we've introduced the Laplacian operator, $\nabla^2 = \frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2}$.

### The finite difference approximation

The key is the Laplacian operator. It is like the $\mathbf{A}$ matrix we used heavily in prior units, but it now contains two derivatives. Let's think about how this works using similar notation to before. A second dimension, $y$, introduces a new index. Now $u_{i,j}^k = u(x_0+i\Delta x, y_0+j\Delta y, t_0 +k\Delta t)$. Note $x\to i$, $y \to j$, and $t \to k$. The Laplacian applied to $u_{i,j}^k$ is

$\nabla^2 u \approx \frac{1}{\Delta x^2} \left(u_{i-1,j}^k - 2 u_{i,j}^k + u_{i+1,j}^k + u_{i,j-1}^k - 2 u_{i,j}^k + u_{i,j+1}^k\right)$

$ = \frac{1}{\Delta x^2} \left(u_{i,j-1}^k + u_{i-1,j}^k - 4 u_{i,j}^k  + u_{i+1,j}^k + u_{i,j+1}^k\right)$

Which, if the column vector $\mathbf{u}$ is written down the rows of a matrix representing the data

While it will prove challenging, you've already learned what you need to know about implementing this in Python. For this homework, you will solve the 2D wave equation both implicitly and explicitly. 

### Hints, tips, and requirements

##### Some words about boundary conditions
In one dimension, we had left and right points as boundary conditions. Now, we've got a more complicated situation with $N$ points on each of 4 boundaries - top, bottom, left, and right. I recommend that you create descriptive labels for each of the boundaries and then apply them in something like the way I left for you to look at in the code below.

##### Move to sparse matrices
I've been sloppy about pushing you to use sparse matrices, but in this assignment I'm going to insist that you make the transition. It isn't difficult. Consult [scipy.spase][https://docs.scipy.org/doc/scipy/reference/sparse.html]. 

##### Matrix operators
For the explicit model, you'll need to find the second derivative of every value of a previous solution. To do this, you need to use an operation like A * u, where A is the second derivative operator matrix, and u is some solution. I think you know how to form the operator matrix. You'll need to do the multiplication. This can be done with A.dot(u). 

##### Visual package
This assignment is going to be a lot more enjoyable if you can look at the output. To do this move to the visual package to represent your output. There is code below. It won't work in Jupyter, but is there for your reference so you can use it from the command line.

##### Problem setup
I've left some code below to help you understand initial conditions and boundary conditions.

##### Specific questions
With your code I want you to address explicit vs. implicit solutions. Determine which is best for
* stability, how big a beta can you use?
* performance, which is faster - be quantitative.
* accuracy, do either of the solutions introduce behaviors that aren't in the physics?

Address these questions with some written text that is submitted along with your code.





In [ ]:
# Explicit Method

from __future__ import division
import numpy as np
import visual as vp
import vpm
import time
from scipy.sparse import csr_matrix

def gaussian(x,y,mu_x = 0,mu_y=0,sigma=.3):
    return np.exp(-(np.sqrt((x-mu_x)**2+(y-mu_y)**2))**2/(2*sigma**2))

L = 1   # world space (?)
N = 25  # Number of mesh points in the x and y directions.
beta = 0.1

# spacial vectors

xr = yr = np.linspace(-L/2, L/2, N)
x, y = np.meshgrid(xr, yr)

# Initial and previous times values of solution
init_value =  gaussian(x,y)
u0 = u1 = init_value

# A matrix
A = np.diag([-4] * N**2) # diagonols, local element
A += np.diag([1] * (N**2 - 1), 1) + np.diag([1] * (N**2 - 1), -1) # left and right
A += np.diag([1] * (N**2 - N), N) + np.diag([1] * (N**2 - N), -N) # top and bottom
I = np.identity(N**2)
I = csr_matrix(I)
A = csr_matrix(A)

# For boundary conditions:
top    = list(range(N))
left   = list(range(N, N**2 - N, N))
right  = list(range(2*N - 1, N**2 - N + 1, N))
bottom = list(range(N**2 - N, N**2))

boundary = top+left+right+bottom

# Set the row to be 1 on the diagonal and 0 elsewhere for boundary elements
A[boundary,:] = I[boundary,:]
    
def wave2d():
    global u0, u1
    
    # The following will turn NxN arrays into column vectors needed for finding products
    u0  = u0.flatten()
    u1 = u1.flatten()

    uN = beta**2 * A.dot(u1) +2 * u1 - u0
    uN[boundary] = 0

    u0 = u1
    u1 = uN
    
    u0 = u0.reshape(N, N)
    u1 = u1.reshape(N, N)
    uN = uN.reshape(N, N)
    return uN

# setup the graphics dispay
scene = vp.display(title='2D waves Explicit', background=(.3,.3,.5), center=(L/2,L/2,-.4), up=(0,0,1), forward=(10,10,-5.0))
mesh = vpm.net(x, y, u0, vp.color.yellow, 0.005)

count = 0
while(True):
    vp.rate(50), vpm.wait(scene)
    uN = wave2d()
    mesh.move(x, y, uN)
    if count%100 == 0:
        print(time.time())
    count += 1


Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days
C:\Users\falkzach\Anaconda3\envs\py27\lib\site-packages\scipy\sparse\compressed.py:739: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


1493265990.94
1493265992.9


In [ ]:
# A sketch of a script for running the simulation with a script and producing nice 3D output

from __future__ import division
import numpy as np, visual as vp, vpm
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix, lil_matrix
import time

# As coded before for setup of matrices and initial values and boundary conditions

def gaussian(x,y,mu_x = 0,mu_y=0,sigma=.1):
    return np.exp(-(np.sqrt((x-mu_x)**2+(y-mu_y)**2))**2/(2*sigma**2))

L = 1   # world space (?)
N = 25  # Number of mesh points in the x and y directions.
beta = 1e-2

# spacial vectors

xr = yr = np.linspace(-L/2, L/2, N)
x, y = np.meshgrid(xr, yr)

# Initial and previous times values of solution
init_value =  gaussian(x,y)
u0 = u1 = init_value

# A matrix
A = np.diag([-4] * N**2) # diagonols, local element
A += np.diag([1] * (N**2 - 1), 1) + np.diag([1] * (N**2 - 1), -1) # left and right
A += np.diag([1] * (N**2 - N), N) + np.diag([1] * (N**2 - N), -N) # top and bottom
I = np.identity(N**2)
I = csr_matrix(I)
A = csr_matrix(A)

# For boundary conditions:
top    = list(range(N))
left   = list(range(N, N**2 - N, N))
right  = list(range(2*N - 1, N**2 - N + 1, N))
bottom = list(range(N**2 - N, N**2))

boundary = top+left+right+bottom

# Set the row to be 1 on the diagonal and 0 elsewhere for boundary elements
A[boundary,:] = I[boundary,:]


def wave2d(u0, u1):
    # an update function, really this is 'all' you have to do
    u0  = u0.flatten()
    u1 = u1.flatten()
    
    Ao = I - ((beta**2 / 2) * A)
    b = ((1/2) * beta**2)*A.dot(u1) + 2 * u1 - u0
    uN = spsolve(Ao, b)
    uN[boundary] = 0
    
    uN = uN.reshape(N, N)
    return uN

scene = vp.display(title='2D waves Crank Nicolson', background=(.3,.3,.5), center=(L/2,L/2,-.4), up=(0,0,1), forward=(10,10,-5.0))
mesh = vpm.net(x, y, u0, vp.color.yellow, 0.005)
count = 0
while(True):
    vp.rate(50), vpm.wait(scene)                  # pause if key press
    uN = wave2d(u0, u1)
    mesh.move(x, y, uN)
    u1, u0 = uN, u1                               # swap u's
    if count%100 == 0:
        print(time.time())
    count += 1

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days
C:\Users\falkzach\Anaconda3\envs\py27\lib\site-packages\scipy\sparse\compressed.py:739: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


1493266196.9
1493266198.87
1493266200.87
1493266202.87
1493266204.88
1493266206.88
1493266208.88
1493266210.88
1493266212.88
1493266214.88
1493266216.88
1493266218.88
1493266220.88
1493266222.88



# Analysis

## Performance:
Crank-Nicolson performes takes a performance hit over the explicite method. In my test the explicite method took on average 1.9823 seconds per 100 iterations while Crank-Nicolson averaged 2.7028 seconds per 100 iterations. This gives the explicit roughly 1.4x ther performance of Crank-Nicolson.

I did not seem to see any performance gain by adding sparce matracies to the explicite implementation, still averaging 1.9952 seconds per 100 iterations (a slight performance hit actually). However with Crank-Nicolson, implementing sparse matracies results in a performance gain at an average of 2.0070 seconds per 100 iterations. This is quite impresive. However I ran into the following Warning:

> SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)
  
So I experimented with it and tried using lil_matrix, the result was a performance hit reducing us to 2.390 seconds per 100 iterations.

## Stability:
The explicit method is dependent on a small beta to not completely deterate. At beta >=0.8, it is unstable on launch. Any beta from 0.3 - 0.8 is pretty chaotic, with the clear progressing steps in the wave caused by the differentiation. This is not a smooth wave and does not accuratly represent the physics. With a beta of 0.1, the early wave forms look similer to that of Crank-Nicolson at a beta of 0.5.

With Crank-Nicolson, a beta of > 0.8 is pretty uninteresting, the surface is very close to rest and bounces gently back and forht. At smaller beta's Crank-Nicolson is very stable. I experimented all the way down to 1e-2 and it is beautifully smooth.

## Conclusion
Crank-Nicolson has the clear stability advantage over the Explicite method. When sparse matraces are implemented, the performance hit for the additional computation involved in Crank-Nicolson is reduced to be equivelent in performance to the explicite method which yielded no gains.
